In [1]:
import sqlite3
import pandas as pd
import os

In [2]:
class DB:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = sqlite3.connect(self.db_path)
        self.c = self.conn.cursor()
        path = "db/scripts/"
        self.setup_scripts_paths = ["create_tables.sql",
                                   "create_views.sql",
                                   "create_triggers.sql"]
        self.setup_scripts_paths = [path + i for i in self.setup_scripts_paths]
        
    def close(self):
        self.conn.close()
        
    
    def remove_db_file(self):
        self.close()
        os.remove(self.db_path)
        
    def run_setup_scripts(self):
        
        for path in self.setup_scripts_paths:
            with open(path) as f:
                query = f.readlines()
                self.c.executescript(''.join(query))
                
                
    def show_tables(self):
        results = self.c.execute("select name from sqlite_master where type = 'table';")
        return results.fetchall()
    
    def select_all(self, table_name):
        results = self.c.execute("select * from {0};".format((table_name)))
        columns = [i[0] for i in self.c.description]
        return columns, results.fetchall()
    
    def _insert_from_data_frame(self, table_name, df):
        df.to_sql(table_name, self.conn, if_exists = "append", index=False)
        
    def _load_initial_data_from_file(self, initial_data_path = "db/initial_data/initial_data.xlsx"):
        initial_data_path = "db/initial_data/initial_data.xlsx"
        sheets_names = ["users", "jobs", "requests", "distances", "cities"]
        
        xls = pd.ExcelFile(initial_data_path)

        self._initial_data = {i: None for i in sheets_names}
        for sheet_name in sheets_names:
            self._initial_data[sheet_name] = xls.parse(sheet_name)
    
    def insert_initial_data(self):
        self._load_initial_data_from_file()
        
        for table_name, data in self._initial_data.items():
            self._insert_from_data_frame(table_name, data)



In [3]:
db = DB("db/test_db.db")
db.setup_scripts_paths

['db/scripts/create_tables.sql',
 'db/scripts/create_views.sql',
 'db/scripts/create_triggers.sql']

In [4]:
db.run_setup_scripts()

In [5]:
db.show_tables()

[('users',),
 ('jobs',),
 ('requests',),
 ('distances',),
 ('results',),
 ('cities',),
 ('execution_logs',),
 ('working_days',)]

In [6]:
db.select_all("cities")

(['city_id', 'city_name'], [])

In [7]:
db._load_initial_data_from_file()
# db._initial_data

In [8]:
db.insert_initial_data()

In [9]:
db.select_all("users")

(['user_id', 'name', 'password', 'admin', 'active', 'time_created'],
 [(1, 'adam', 'pass1', 0, 1, '2019-10-10 00:00:00'),
  (2, 'tom', 'pass2', 0, 1, '2019-10-10 00:00:00'),
  (3, 'admin', 'admin', 1, 1, '2019-10-10 00:00:00')])

In [10]:
db.close()

In [11]:
db.remove_db_file()